In [40]:
#! pip install langchain==0.3.26
#! pip install flask==3.1.1 
#! pip install sentence-transformers==4.1.0
#! pip install pypdf==5.6.1 
#! pip install python-dotenv==1.1.0
#! pip install langchain-pinecone==0.2.8 
#! pip install langchain-openai==0.3.24
#! pip install langchain-community==0.3.26
#! pip install dotenv

In [2]:
%pwd

'/Users/hassanfarid/Documents/Complete-Medical-Chatbot-with-LLMs-LangChain-Pinecone-Flask-AWS/research'

In [3]:
import os

os.chdir('../')

In [4]:
%pwd

'/Users/hassanfarid/Documents/Complete-Medical-Chatbot-with-LLMs-LangChain-Pinecone-Flask-AWS'

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

/opt/anaconda3/envs/medibot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob = "*.pdf",
        loader_cls = PyPDFLoader
    )
    documents = loader.load()
    return documents

In [17]:
data_path = 'Dataset/'
extracted_data = load_pdf_files(data_path)

In [25]:
extracted_data[100]

Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Dataset/Medical_book.pdf', 'total_pages': 637, 'page': 100, 'page_label': '101'}, page_content='• T cell lymphocytes: 644-2200/mm\n3\n, 60-88% of all lym-\nphocytes.\n• B cell lymphocytes: 82-392/mm\n3\n, 3-20% of all lympho-\ncytes.\n• CD4+ lymphocytes: 500-1200/mm\n3\n, 34-67% of all\nlymphocytes.\nAbnormal results\nThe following results in AIDS tests indicate progres-\nsion of the disease:\n• Percentage of CD4+ lymphocytes: less than 20% of all\nlymphocytes.\n• CD4+ lymphocyte count: less than 200 cells/mm\n3\n.\n• Viral load test: Levels more than 5000 copies/mL.\n• /H9252-2-microglobulin: Levels more than 3.5 mg/dL.\n• P24 antigen: Measurable amounts in blood serum.\nResources\nBOOKS\nAvrameas, Stratis, and Therese Ternynck. “Enzyme Linked\nImmunosorbent Assay (ELISA).” In Encyclopedia of\nImmunology.V ol. 1.

In [ ]:
print("Length of data(pages):",len(extracted_data)) 

Length of data(pages) 637


In [27]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only "source" in the metadata and the original page_content.
    """
    
    minimal_docs: List[Document] = []
    
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content = doc.page_content,
                metadata = {"source": src}
            )
        )
    return minimal_docs

In [28]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [29]:
minimal_docs[100]

Document(metadata={'source': 'Dataset/Medical_book.pdf'}, page_content='• T cell lymphocytes: 644-2200/mm\n3\n, 60-88% of all lym-\nphocytes.\n• B cell lymphocytes: 82-392/mm\n3\n, 3-20% of all lympho-\ncytes.\n• CD4+ lymphocytes: 500-1200/mm\n3\n, 34-67% of all\nlymphocytes.\nAbnormal results\nThe following results in AIDS tests indicate progres-\nsion of the disease:\n• Percentage of CD4+ lymphocytes: less than 20% of all\nlymphocytes.\n• CD4+ lymphocyte count: less than 200 cells/mm\n3\n.\n• Viral load test: Levels more than 5000 copies/mL.\n• /H9252-2-microglobulin: Levels more than 3.5 mg/dL.\n• P24 antigen: Measurable amounts in blood serum.\nResources\nBOOKS\nAvrameas, Stratis, and Therese Ternynck. “Enzyme Linked\nImmunosorbent Assay (ELISA).” In Encyclopedia of\nImmunology.V ol. 1. Ed. Ivan M. Roitt and Peter J. Delves.\nLondon: Academic Press, 1992.\nBennett, Rebecca, and Erin, Charles A. (Editors). HIV and\nAIDS Testing, Screening, and Confidentiality: Ethics, Law,\nand Soci

In [32]:
def text_split(docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,
        chunk_overlap = 20
    )
    text_chunks = text_splitter.split_documents(docs)
    return text_chunks

In [33]:
text_chunks = text_split(minimal_docs)
print(f'Number of Chunks: {len(text_chunks)}')

Number of Chunks: 5859


In [35]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return huggingface embedding model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name = model_name
    )
    return embeddings
    
embedding = download_embeddings()

/var/folders/fh/3l8bx7nj5tv9qzbx13sctjjm0000gn/T/ipykernel_20097/4289267064.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [49]:
print(embedding)
vectors = embedding.embed_query("Hello, World")

print(f"Vectors: {vectors}")
print(f"Length of Vectors: {len(vectors)}")

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False
Vectors: [-0.056094396859407425, 0.03554259240627289, 0.004592870362102985, 0.02386585623025894, -0.04944375529885292, -0.15514132380485535, 0.06592568755149841, 0.022491320967674255, -0.021727290004491806, 0.014119270257651806, 0.05505150556564331, 0.024055687710642815, 0.005019017029553652, -0.006477480288594961, -0.03411555662751198, -0.05552103742957115, -0.0067526232451200485, -0.02301395684480667, -0.

In [42]:
from dotenv import load_dotenv
load_dotenv()

True

In [44]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [46]:
from pinecone import Pinecone
pinecine_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecine_api_key)

In [47]:
pc

In [53]:
from pinecone import ServerlessSpec

index_name = "medical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension = 384,
        metric = 'cosine',
        spec = ServerlessSpec(cloud='aws', region= 'us-east-1')
    )
    
index = pc.Index(index_name)

In [55]:
from langchain_pinecone import PineconeVectorStore

doc_search = PineconeVectorStore.from_documents(
    documents = text_chunks,
    embedding = embedding,
    index_name = index_name
)

In [56]:
# Load Existing Index

from langchain_pinecone import PineconeVectorStore

doc_search = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding = embedding,
    
)

In [60]:
# Add more data to the existing Pinecon Index

add_more = Document(
    page_content = " I am M Hassan Farid, Lab Engineer at Abasyn University.",
    metadata = {'source': 'LinkedIN'}
)

# doc_search.add_documents(documents=[add_more])

['93f28693-a54c-48c7-a6fe-6b0aa84c5179']

In [61]:
# Retriever

retriever = doc_search.as_retriever(search_type='similarity', search_kwargs={"k":3})

In [ ]:
retriev_docs = retriever.invoke("What is Acne?")
retriev_docs

[Document(id='54dfbdeb-db0b-4a1d-91d9-5c4895b3228e', metadata={'source': 'Dataset/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='de95ed55-45f6-465d-aa74-7f27f3910cd3', metadata={'source': 'Dataset/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='6f9ba235-a455-4a71-8f1e-a6bdb5f56256', metadata={'source': 'Dataset/Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clog

In [64]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(
    model = 'gpt-4o'
)

In [68]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [70]:
# System Prompt

system_prompt = (
    "You are an Medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ('system',system_prompt),
        ('human', '{input}')
    ]
)

In [ ]:
question_answere_chain = create_stuff_documents_chain(chat_model,prompt)
rag_chain = create_retrieval_chain(retriever, question_answere_chain)

In [73]:
response = rag_chain.invoke({"input": 'What is Acromegaly and gigantism?'})
response

{'input': 'What is Acromegaly and gigantism?',
 'context': [Document(id='e62b0e54-080b-430c-83d5-f4489d09e612', metadata={'source': 'Dataset/Medical_book.pdf'}, page_content='Whitehouse Station, NJ: Merck Research Laboratories,\n1997.\nLarsen, D. E., ed. Mayo Clinic Family Health Book.New York:\nWilliam Morrow and Co., Inc., 1996.\nJohn T. Lohr, PhD\nAcromegaly and gigantism\nDefinition\nAcromegaly is a disorder in which the abnormal\nrelease of a particular chemical from the pituitary gland\nin the brain causes increased growth in bone and soft tis-\nsue, as well as a variety of other disturbances throughout\nthe body. This chemical released from the pituitary gland'),
  Document(id='f0c76072-89a7-488e-b598-28db1355c377', metadata={'source': 'Dataset/Medical_book.pdf'}, page_content='mone Excess: Acromegaly and Gigantism.” In Harrison’s\nPrinciples of Internal Medicine, ed. Anthony S. Fauci, et\nal. New York: McGraw-Hill, 1997.\nJameson, J. Larry. “Growth Hormone Excess: Acromegaly an

In [75]:
print(response['answer'])

Acromegaly is a disorder caused by the abnormal release of a chemical from the pituitary gland, leading to increased growth in bone and soft tissue. Gigantism, also related to excess growth hormone, occurs when the abnormality begins before bone growth stops, resulting in unusual height. Both conditions involve disturbances caused by hormone imbalances in the body.


In [76]:
response = rag_chain.invoke({'input': 'What is Acne?'})
print(response['answer'])

Acne is a common skin disease characterized by pimples on the face, chest, and back, occurring when the skin pores become clogged with oil, dead skin cells, and bacteria. It is medically known as acne vulgaris and is the most common skin disease, affecting nearly 17 million people in the United States.


In [77]:
response = rag_chain.invoke({'input':'What is the treatment of Acne?'})
print(response['answer'])

The treatment of acne depends on its severity. For mild non-inflammatory acne, topical treatments like tretinoin, benzoyl peroxide, adapalene, or salicylic acid are used, often with improvement seen in two to four weeks. For inflammatory acne, topical antibiotics may be added, and in more severe cases, oral medications such as isotretinoin may be considered.
